In [71]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [72]:
import pandas as pd
from functools import reduce
import os

In [85]:
if os.path.exists("../rawdata/ship_df.csv"):
    ship_df = pd.read_csv("../rawdata/ship_df.csv")
else:
    file_columns = {
        "fuelDensity.csv": ["timestamp", "fuel_density"],
        "fuelTemp.csv": ["timestamp", "fuel_temp"],
        "fuelVolumeFlowRate.csv": ["timestamp", "fuel_volume_flow_rate"],
        "inclinometer-raw.csv": ["timestamp", "inclinometer_raw"],
        "latitude.csv": ["timestamp", "latitude"],
        "level1median.csv": ["timestamp", "level1_median"],
        "level2median.csv": ["timestamp", "level2_median"],
        "longitude.csv": ["timestamp", "longitude"],
        "longitudinalWaterSpeed.csv": ["timestamp", "longitudinal_water_speed"],
        "portPitch.csv": ["timestamp", "port_pitch"],
        "portRudder.csv": ["timestamp", "port_rudder"],
        "speedKmh.csv": ["timestamp", "speed_kmh"],
        "speedKnots.csv": ["timestamp", "speed_knots"],
        "starboardPitch.csv": ["timestamp", "starboard_pitch"],
        "starboardRudder.csv": ["timestamp", "starboard_rudder"],
        "trackDegreeMagnetic.csv": ["timestamp", "track_degree_magnetic"],
        "trackDegreeTrue.csv": ["timestamp", "track_degree_true"],
        "trueHeading.csv": ["timestamp", "true_heading"],
        "windAngle.csv": ["timestamp", "wind_angle"],
        "windSpeed.csv": ["timestamp", "wind_speed"],
    }

    dfs = [
        pd.read_csv(f"../rawdata/{file}", header=None, names=columns)
        for file, columns in file_columns.items()
    ]

    ship_df = reduce(lambda left, right: pd.merge(left, right, on="timestamp", how="outer"), dfs)
    ship_df.to_csv("../rawdata/ship_df.csv", index=False)


In [86]:
if "converted_timestamp" not in ship_df.columns:
    def convert_dotnet_timestamp(ticks):
        unix_timestamp = (ticks // 10_000_000) - 62135596800
        return pd.to_datetime(unix_timestamp, unit='s')

    ship_df['converted_timestamp'] = ship_df['timestamp'].apply(convert_dotnet_timestamp)
    ship_df.to_csv("../rawdata/ship_df.csv", index=False)

In [87]:
ship_df.drop("timestamp", axis=1, inplace=True)

In [88]:
# Matches dataset description in assignment
ship_df['converted_timestamp'].min(), ship_df['converted_timestamp'].max()

('2010-02-15 05:45:12', '2010-04-12 22:25:16')

In [89]:
(ship_df.isna().sum() / len(ship_df)).sort_values(ascending=False)

longitudinal_water_speed    0.971211
wind_speed                  0.954648
wind_angle                  0.954648
true_heading                0.924416
fuel_density                0.914556
fuel_temp                   0.914556
fuel_volume_flow_rate       0.914556
starboard_rudder            0.909371
starboard_pitch             0.909371
port_rudder                 0.909371
port_pitch                  0.909371
track_degree_magnetic       0.907250
track_degree_true           0.907249
speed_kmh                   0.907227
speed_knots                 0.907227
longitude                   0.859405
latitude                    0.859404
inclinometer_raw            0.813096
level2_median               0.798335
level1_median               0.791038
converted_timestamp         0.000000
dtype: float64

In [90]:
ship_df.sort_values("converted_timestamp", inplace=True)

In [91]:
ship_df.dtypes

fuel_density                float64
fuel_temp                   float64
fuel_volume_flow_rate       float64
inclinometer_raw            float64
latitude                     object
level1_median               float64
level2_median               float64
longitude                    object
longitudinal_water_speed    float64
port_pitch                  float64
port_rudder                 float64
speed_kmh                   float64
speed_knots                 float64
starboard_pitch             float64
starboard_rudder            float64
track_degree_magnetic       float64
track_degree_true           float64
true_heading                float64
wind_angle                  float64
wind_speed                  float64
converted_timestamp          object
dtype: object

In [92]:
def coords_to_decimal(coord):
    if not isinstance(coord, str):  # Handle NaNs or unexpected values
        return None

    direction = coord[-1]  # Last character (N, S, E, W)
    value = coord[:-1]  # Remove direction (00649.1143)

    # Extract degrees and minutes
    degrees = int(value[:-5])  # Everything before the last 5 digits
    minutes = float(value[-5:])  # Last 5 digits as minutes

    # Convert to decimal degrees
    decimal = degrees + (minutes / 60)

    # Apply negative sign for South and West
    if direction in ['S', 'W']:
        decimal *= -1

    return decimal

# Apply conversion
ship_df['latitude'] = ship_df['latitude'].apply(coords_to_decimal)
ship_df['longitude'] = ship_df['longitude'].apply(coords_to_decimal)


In [93]:
ship_df.dtypes

fuel_density                float64
fuel_temp                   float64
fuel_volume_flow_rate       float64
inclinometer_raw            float64
latitude                    float64
level1_median               float64
level2_median               float64
longitude                   float64
longitudinal_water_speed    float64
port_pitch                  float64
port_rudder                 float64
speed_kmh                   float64
speed_knots                 float64
starboard_pitch             float64
starboard_rudder            float64
track_degree_magnetic       float64
track_degree_true           float64
true_heading                float64
wind_angle                  float64
wind_speed                  float64
converted_timestamp          object
dtype: object

In [94]:
print("Before grouping: ", ship_df.shape)

# Ensure 'converted_timestamp' is in datetime format
ship_df['converted_timestamp'] = pd.to_datetime(ship_df['converted_timestamp'], unit='ns')

# Create a new column for grouping
ship_df['grouped_timestamp'] = ship_df['converted_timestamp'].dt.floor('min')

# Group by the new column, then drop it if not needed
ship_df = ship_df.groupby('grouped_timestamp').mean().reset_index()

ship_df.drop('grouped_timestamp', axis=1, inplace=True)

print("After grouping: ", ship_df.shape)


Before grouping:  (19045464, 21)
After grouping:  (6031, 21)


In [95]:
ship_df.head(1000)

,fuel_density,fuel_temp,fuel_volume_flow_rate,inclinometer_raw,latitude,level1_median,level2_median,longitude,longitudinal_water_speed,port_pitch,...,speed_kmh,speed_knots,starboard_pitch,starboard_rudder,track_degree_magnetic,track_degree_true,true_heading,wind_angle,wind_speed,converted_timestamp
0,NaN,NaN,NaN,357.597872,6132.602083,16.653623,14.176539,-648.555176,7.952747,5.117690,...,15.433898,8.330169,5.249923,-1.323209,167.945578,175.639456,160.238075,230.391608,4.060140,2010-02-15 05:47:35.727365376
1,NaN,NaN,NaN,357.281812,6132.007593,16.576844,14.467348,-646.405858,17.597895,7.568833,...,33.040523,17.839869,7.596906,0.110833,131.302288,124.302288,129.574600,26.000000,8.702667,2010-02-15 05:52:29.738204416
2,NaN,NaN,NaN,357.235615,6131.681677,16.841274,14.319438,-643.653420,18.536842,7.805097,...,35.266124,19.043974,7.753260,0.188460,79.836156,72.836156,70.224000,148.113333,9.710000,2010-02-15 05:57:29.071451904
3,NaN,NaN,NaN,357.187994,6132.771044,17.046858,14.363045,-642.053399,18.544211,7.790150,...,36.607869,19.759344,7.833700,-0.064570,24.653770,17.653770,9.188000,246.740000,8.806667,2010-02-15 06:02:29.427772160
4,NaN,NaN,NaN,357.222411,6134.411845,17.064005,14.379844,-640.929260,18.734375,7.804730,...,35.953247,19.418831,7.846677,-0.065110,25.670779,18.670779,9.824600,52.206667,8.268667,2010-02-15 06:07:29.403103232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.927119,96.027643,0.680634,357.258188,6140.371473,16.436549,15.266279,-648.371686,19.894737,8.611380,...,37.489577,20.242020,8.544523,0.055663,343.189577,336.189577,352.685320,27.420000,29.014000,2010-02-24 07:32:29.230745856
996,0.927074,96.090529,0.687897,357.288786,6141.769021,16.358588,15.269014,-649.669586,19.994737,8.612538,...,37.156066,20.075410,8.537957,0.067816,343.172787,336.172787,351.698800,26.886667,28.938667,2010-02-24 07:37:29.109057280
997,0.926891,96.199618,0.680009,357.261828,6143.413518,16.395881,15.201623,-651.189120,20.222917,8.685953,...,37.669508,20.335082,8.608000,0.046513,343.872787,336.872787,352.278600,29.220000,24.104667,2010-02-24 07:42:29.647451392
998,0.926805,96.266460,0.680979,357.290000,6144.903463,16.369800,15.158440,-652.525458,20.096842,8.635000,...,37.548701,20.274351,8.562907,0.006950,345.313961,338.313961,354.479080,17.600000,29.077333,2010-02-24 07:47:29.652701952


In [96]:
(ship_df.isna().sum() / len(ship_df)).sort_values(ascending=False)

fuel_density                0.036810
fuel_volume_flow_rate       0.036810
fuel_temp                   0.036810
level2_median               0.002487
inclinometer_raw            0.000829
port_pitch                  0.000663
starboard_rudder            0.000663
starboard_pitch             0.000663
port_rudder                 0.000663
longitudinal_water_speed    0.000332
level1_median               0.000332
wind_angle                  0.000332
wind_speed                  0.000332
speed_kmh                   0.000166
speed_knots                 0.000166
track_degree_magnetic       0.000166
track_degree_true           0.000166
longitude                   0.000000
latitude                    0.000000
true_heading                0.000000
converted_timestamp         0.000000
dtype: float64